<a href="https://colab.research.google.com/github/Chood16/DSCI222/blob/main/lectures/(11)_Image__Processing_with_OpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Processing with OpenCV

## 1. Image Basics

In [ ]:
import cv2  # For Image Processing
import numpy as np # Numpy
from google.colab.patches import cv2_imshow # For Google Colab


In [ ]:
arr = np.array([
    [[100, 0, 0],   [0, 250, 0]],
    [[0, 0, 255],  [150, 50, 125]]
], dtype=np.uint8)

# To display the image

# *** The normal approach
#cv2.imshow('image', image_orig)  # The normal approach
#cv2.waitKey() # This is necessary to be required so that the image doesn't close immediately.
#It will run continuously until the key press.
#cv2.destroyAllWindows()

# *** For Google Colab
cv2_imshow(arr)


In [ ]:
bigger = cv2.resize(arr, (200, 200))

cv2_imshow(bigger)

In [ ]:
bigger = cv2.resize(arr, (200, 200), interpolation=cv2.INTER_NEAREST)

cv2_imshow(bigger)

In [ ]:
# To show some information about the image
print('Type of object: ')
display(type(bigger))

print('\nShape of the object/image: ')
display( bigger.shape )


What are the maximum and minimum RGB contributions?

`arr = np.array([
    [[100, 0, 0],   [0, 250, 0]],
    [[0, 0, 255],  [150, 50, 125]]
], dtype=np.uint8)`

In [ ]:
print(arr.max())


In [ ]:
# Collapsing rows
print(arr.max(axis=0))



In [ ]:
# Collapsing columns
print(arr.max(axis=1))


In [ ]:
# Collapsing rows and columns
print(arr.max(axis=(0,1)))


In [ ]:
# For grayscale
print('\nMinimum value: ')
display( np.min(bigger) )
print('\nMaximum value: ')
display( np.max(bigger) )

Let's go back, are the results what we expected?

In [ ]:
arr = np.array([
    [[100, 0, 0],   [0, 250, 0]],
    [[0, 0, 255],  [150, 50, 125]]
], dtype=np.uint8)
bigger = cv2.resize(arr, (200, 200), interpolation=cv2.INTER_NEAREST)

cv2_imshow(bigger)


In [ ]:

#https://docs.opencv.org/4.x/d4/da8/group__imgcodecs.html

image_rgb = cv2.cvtColor(bigger, cv2.COLOR_BGR2RGB)
cv2_imshow( image_rgb )


## 2. Loading and Displaying Images

In [ ]:
import cv2  # For Image Processing
import numpy as np # Numpy
from google.colab.patches import cv2_imshow # For Google Colab
path_img = "Stadium.jpg"

# Using imread('path') and 1 denotes read as  color image
image_orig = cv2.imread(path_img, 1)

cv2_imshow(image_orig)

# To open the original image as gray scale
image_orig_gray = cv2.imread(path_img, 0)
cv2_imshow(image_orig_gray)


In [ ]:
print('\nShape of the object/image: ')
display(image_orig.shape)

# (rows, columns, channels)

In [ ]:
display(image_orig)

In [ ]:
image_rgb = cv2.cvtColor(image_orig, cv2.COLOR_BGR2RGB)
cv2_imshow( image_rgb )

In [ ]:
# To show each channel

# B
image_cpy = image_orig.copy()
image_cpy[:,:,[1,2]] = 0 # To keep B
cv2_imshow(image_cpy)

# G
image_cpy = image_orig.copy()
image_cpy[:,:,[0,2]] = 0 # To keep G
cv2_imshow(image_cpy)

# R
image_cpy = image_orig.copy()
image_cpy[:,:,[0,1]] = 0 # To keep R
cv2_imshow(image_cpy)


##3. Affine Transformation

Take x,y coordinate --> Do Math --> get new x',y' coordinates

https://docs.opencv.org/3.4/d4/d61/tutorial_warp_affine.html

In [ ]:
# Get image height, width
(h, w) = image_orig.shape[:2]
# To compute the center of the image
center = (w / 2, h / 2)

print('Center of the image:')
display(center)



In [ ]:
# What is this M output?
M = cv2.getRotationMatrix2D(center, angle=45, scale=.6)
print(M)


The function returns a $2 \times 3$ affine transformation matrix:

$M =
\begin{bmatrix}
\alpha & \beta & (1-\alpha)\cdot center_x - \beta \cdot center_y \\
-\beta & \alpha & \beta \cdot center_x + (1-\alpha)\cdot center_y
\end{bmatrix}$

where

$\alpha = scale \cdot \cos(\theta)$,

$\beta = scale \cdot \sin(\theta)$


and $\theta$ is the rotation angle.


In [ ]:
# How does warpAffine work?
image_rotated45 = cv2.warpAffine(image_orig, M, (w, h))



Each output point $(x', y')$ is calculated from the input point $(x, y)$ using the affine matrix $M$:

$\begin{bmatrix}
x' \\
y'
\end{bmatrix}
=
\begin{bmatrix}
m_{11} & m_{12} & m_{13} \\
m_{21} & m_{22} & m_{23}
\end{bmatrix}
\cdot
\begin{bmatrix}
x \\
y \\
1
\end{bmatrix}$

where $M$ is the $2 \times 3$ affine transformation matrix:

$
M =
\begin{bmatrix}
m_{11} & m_{12} & m_{13} \\
m_{21} & m_{22} & m_{23}
\end{bmatrix}
$


In [ ]:
cv2_imshow(image_rotated45)

## 4. Canny Edge Detector

Used in Object Recognition

https://en.wikipedia.org/wiki/Canny_edge_detector

https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html

How do we determine, where edges are?

`threshold1` (lower threshold)

* Pixels with gradient below this value are discarded.

* Pixels with gradient between threshold1 and threshold2 are kept only if connected to a strong edge.

* These are called weak edges.

`threshold2` (upper threshold)

* Pixels with gradient above this value are considered strong edges and always kept.

Lower thresholds → detect more edges (even faint ones), but may include noise.

Higher thresholds → detect only strong edges (less noise, but may miss subtle edges).

`L2gradient`
* `True` uses Euclidean Distance
* `False` uses Taxicab Distance

In [ ]:
threshold1 = 200
threshold2 = 300
L2gradient = True
image_edges = cv2.Canny(image_orig, threshold1, threshold2, L2gradient)
cv2_imshow(image_edges)

In [ ]:
# This works for images of individuals too!
path_img = 'Favorite_Professor.jpeg'

# Using imread('path') and 1 denotes read as  color image
image_face = cv2.imread(path_img, 1)

# To display the new image
cv2_imshow(image_face)

# Canny Edge Detection
image_edges_lady = cv2.Canny(image_face, 100, 200, True)
cv2_imshow(image_edges_lady)

## 5. Bilateral Filter (Image Smoothing)
https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html

`Diameter`  (How big of a brush to use)
* Represents pixel neighborhood size
* Large value = large smoothing area

`sigmaColor` (Amount of smoother)
* Determines how different a pixel’s color can be to be averaged.
* Small sigmaColor (e.g., 25–50): Only very similar colors are averaged → preserves more texture, less smoothing.
* Large sigmaColor (e.g., 75–150): Larger differences are included → smoother skin, may wash out subtle features.

`sigmaSpace` (Where smoothing occurs)
* Determines how far in the image space the filter looks.
* Small sigmaSpace (e.g., 25–50): Only nearby pixels influence averaging → fine detail preserved.
* Large sigmaSpace (e.g., 75–150): Pixels farther away contribute → stronger smoothing over wider areas.

In [ ]:
cv2_imshow(image_face)

# After bilateral filter
d = 12
sigmaColor = 55
sigmaSpace = 55
image_face_bilat = cv2.bilateralFilter(image_face, d, sigmaColor, sigmaSpace)
cv2_imshow(image_face_bilat)

We can blurr images too!

In [ ]:
# Apply Gaussian blur directly on the color image
kernal = (11,11) # the "window size" used for blurring each pixel
SigmaX = 0 # how much close vs far neighbors impact each pixel.
# 0 means we don't calculate it, OpenCV does

blurred_color = cv2.GaussianBlur(image_face, kernal, SigmaX)

# Display the blurred color image
cv2_imshow(image_face)
cv2_imshow(blurred_color)

## 6. Image Matching

In [ ]:
# Let's see if we can find he following person in the image

# Original template
i_spy = cv2.imread('Hidden_Flag.jpeg') # Flag, Person
cv2_imshow(i_spy)

# The method we will be using requires gray-scale images
i_spy_gray = cv2.imread('Hidden_Flag.jpeg', 0)
cv2_imshow(i_spy_gray)

In [ ]:
# Here is a reminder of the image we are searching

# To display the original image
cv2_imshow(image_orig)
# To open the original image as gray scale
path_img = "Stadium.jpg"
image_orig_gray = cv2.imread(path_img, 0)
cv2_imshow(image_orig_gray)

How does matching in OpenCV work?

`cv2.matchTemplate` slides the template image (`i_spy_gray`) over the source image (`image_orig_gray`) like a “window” and computes a similarity score at each position.

Think of it as checking how well the template matches each part of the big image.

The result is a 2D matrix res where each element corresponds to the matching score at a specific ***top-left coordinate*** in the source image.

This can feel like facial recognition, but is not nearly as robust. The image in the target and source need to be almost identical

In [ ]:
# MATCHING
# To Find the location of the template within the BIG image
w, h = i_spy_gray.shape[::-1]
# Apply template Matching
res = cv2.matchTemplate(image_orig_gray, i_spy_gray, eval('cv2.TM_CCOEFF'))
print(res)

In [ ]:
# For the cv2.TM_CCOEFF evaluation methos, larger is better
# If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

# Let's make a red rectangle around the image that was found
top_left = max_loc
bottom_right = (top_left[0] + w, top_left[1] + h)
color_rec = (0, 0, 255) # Remember: B G R order
thickness = 3

# Draw rectangle on image
cv2.rectangle(image_orig, top_left, bottom_right, color_rec, thickness)
cv2_imshow(image_orig)